In [1]:
# Imports
import pathlib

import numpy as np
import torch
from skimage.io import imread
from skimage.transform import resize

from inference import predict
from transformations import normalize_01, re_normalize
from unet import UNet

# root directory
root = pathlib.Path.cwd() / 'dataset' / 'Test'
# root = pathlib.Path.cwd() / 'Test' 
# root = pathlib.Path.cwd() / 'TheYoung'

def get_filenames_of_path(path: pathlib.Path, ext: str = '*'):
    """Returns a list of files in a directory/path. Uses pathlib."""
    filenames = [file for file in path.glob(ext) if file.is_file()]
    return filenames

# input and target files
images_names = get_filenames_of_path(root / 'Input')
targets_names = get_filenames_of_path(root / 'Target')

# read images and store them in memory
images = [imread(img_name) for img_name in images_names]

for img_name in images_names:
    print(img_name)

targets = [imread(tar_name) for tar_name in targets_names]


for img_name in targets_names:
    print(img_name)

# Resize images and targets
images_res = [resize(img, (128, 128, 3)) for img in images]
resize_kwargs = {'order': 0, 'anti_aliasing': False, 'preserve_range': True}
targets_res = [resize(tar, (128, 128), **resize_kwargs) for tar in targets]

# device
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    torch.device('cpu')



# model
model = UNet(in_channels=3,
             out_channels=2,
             n_blocks=4,
             start_filters=32,
             activation='relu',
             normalization='batch',
             conv_mode='same',
             dim=2).to(device)


model_name = 'lr0001epoch200.pt'
model_weights = torch.load(model_name)

model.load_state_dict(model_weights)

# preprocess function
def preprocess(img: np.ndarray):
    img = np.moveaxis(img, -1, 0)  # from [H, W, C] to [C, H, W]
    img = normalize_01(img)  # linear scaling to range [0-1]
    img = np.expand_dims(img, axis=0)  # add batch dimension [B, C, H, W]
    img = img.astype(np.float32)  # typecasting to float32
    return img


# postprocess function
def postprocess(img: torch.tensor):
    img = torch.argmax(img, dim=1)  # perform argmax to generate 1 channel
    img = img.cpu().numpy()  # send to cpu and transform to numpy.ndarray
    img = np.squeeze(img)  # remove batch dim and channel dim -> [H, W]
    img = re_normalize(img)  # scale it to the range [0-255]
    return img

# predict the segmentation maps 
output = [predict(img, model, preprocess, postprocess, device) for img in images_res]

/home/mianzhi/Desktop/Carnava/dataset/Test/Input/a10033.png
/home/mianzhi/Desktop/Carnava/dataset/Test/Input/a10004.png
/home/mianzhi/Desktop/Carnava/dataset/Test/Input/a10010.png
/home/mianzhi/Desktop/Carnava/dataset/Test/Input/a10009.png
/home/mianzhi/Desktop/Carnava/dataset/Test/Input/a10005.png
/home/mianzhi/Desktop/Carnava/dataset/Test/Input/a10002.png
/home/mianzhi/Desktop/Carnava/dataset/Test/Input/a10003.png
/home/mianzhi/Desktop/Carnava/dataset/Test/Input/a10019.png
/home/mianzhi/Desktop/Carnava/dataset/Test/Input/a10001.png
/home/mianzhi/Desktop/Carnava/dataset/Test/Input/a10006.png
/home/mianzhi/Desktop/Carnava/dataset/Test/Input/a10008.png
/home/mianzhi/Desktop/Carnava/dataset/Test/Input/a10089.png
/home/mianzhi/Desktop/Carnava/dataset/Test/Input/a10007.png
/home/mianzhi/Desktop/Carnava/dataset/Test/Target/a10033.png
/home/mianzhi/Desktop/Carnava/dataset/Test/Target/a10004.png
/home/mianzhi/Desktop/Carnava/dataset/Test/Target/a10010.png
/home/mianzhi/Desktop/Carnava/dataset

In [3]:
import napari

viewer = napari.Viewer()

idx = 3
img_nap = viewer.add_image(images_res[idx], name='Input')
tar_nap = viewer.add_labels(targets_res[idx], name='Target')
out_nap = viewer.add_labels(output[idx], name='Prediction')